# **Non Linear Algorithms**

In [9]:
# Linear Regression With Stochastic Gradient Descent for Wine Quality
from random import seed
from random import randrange
from csv import reader
from math import sqrt

In [10]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if(len(row)>4 and len([val for val in row if '\\' in val])>0):
                row = [val.replace('\\','') for val in row]
            if not row or len(row)<4 or (len([val for val in row if 
                                              ('apple' in val or 'outl0strokewidth0' in val or '}' in val)])>0):
                continue
            dataset.append(row)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
    
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores



# **Classification and Regression Trees**

A node represents a single input variable (X) and a split point on that variable, assuming the
variable is numeric. The leaf nodes (also called terminal nodes) of the tree contain an output
variable (y) which is used to make a prediction. Once created, a tree can be navigated with a
new row of data following each branch with the splits until a final prediction is made.

Creating a binary decision tree is actually a process of dividing up the input space. A greedy
approach is used called recursive binary splitting. This is a numerical procedure where all the
values are lined up and diﬀerent split points are tried and tested using a cost function. The split
with the best cost (lowest cost because we minimize cost) is selected. All input variables and all
possible split points are evaluated and chosen in a greedy manner based on the cost function.

* Regression: The cost function that is minimized to choose split points is the sum squared
error across all training samples that fall within the rectangle.

* Classification: The Gini cost function is used which provides an indication of how pure
the nodes are, where node purity refers to how mixed the training data assigned to each
node is.

## Gini index calculation

Two groups are formed by the split on an attribute given by an index over a value, say A and B.

lets say we have two classes (0 and 1) in our data and both the groups may contain the data from both the classes.

A0 = # of rows in A from class 0

A1 = # of rows in A from class 1

B0 = # of rows in B from class 0

B1 = # of rows in B from class 1



--p is proportion

pA0 = count(A0)/count(A);

pA1 = count(A1)/count(A); 

pB0 = count(B0)/count(B); 

pB1 = count(B1)/count(B); 


gini_index_A = sum_over_classes (p_class (1-p_class)) = 1 - sum_over_class(p_class^2)

gini_index_B = sum_over_classes (p_class (1-p_class)) = 1 - sum_over_class(p_class^2)



final gini index of split is the weighted gini_index as per the group count.

wA = count(A)/total_instances;

wB = count(B)/total_instances;


finally, gini index for a split on column - col[index] over a value is-

GINI_INDEX = wA * (gini_index_A) + wB * (gini_index_B)


In [102]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

# test Gini values
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(gini_index([[[1, 0], [1
                            , 0]], [[1, 1], [1, 1]]], [0, 1]))

0.5
0.0


###  Splitting a dataset
Splitting a dataset means separating a dataset into two lists of rows given the index of an
attribute and a split value for that attribute. Once we have the two groups, we can then use
our Gini score above to evaluate the cost of the split. Splitting a dataset involves iterating over
each row, checking if the attribute value is below or above the split value and assigning it to the
left or right group respectively. Below is a function named test split() that implements this
procedure.

In [14]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

### Evaluating All Splits
With the Gini function above and the test split function we now have everything we need to
evaluate splits. Given a dataset, we must check every value on each attribute as a candidate
split, evaluate the cost of the split and find the best possible split we could make. Once the
best split is found, we can use it as a node in our decision tree.

This is an exhaustive and greedy algorithm. We will use a dictionary to represent a node in
the decision tree as we can store data by name. When selecting the best split and using it as a
new node for the tree we will store the index of the chosen attribute, the value of that attribute
by which to split and the two groups of data split by the chosen split point.
Each group of data is its own small dataset of just those rows assigned to the left or right
group by the splitting process. We might split each group again, recursively
as we build out our decision tree.

In [19]:
# Select the best split point for a dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            #print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [132]:
# Test getting the best split
dataset = [[2.771244718,1.784783929,0],
[1.728571309,1.169761413,0],
[3.678319846,2.81281357,0],
[3.961043357,2.61995032,0],
[2.999208922,2.209014212,0],
[7.497545867,3.162953546,1],
[9.00220326,3.339047188,1],
[7.444542326,0.476683375,1],
[10.12493903,3.234550982,1],
[6.642287351,3.319983761,0]]
split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

Split: [X1 < 7.445]


## Build a Tree

#### Terminal Nodes
We need to decide when to stop growing a tree. We can do that using the depth and the number
of rows that the node is responsible for in the training dataset.

* Maximum Tree Depth. This is the maximum number of nodes from the root node of
the tree. Once a maximum depth of the tree is met, we must stop adding new nodes.
Deeper trees are more complex and are more likely to overfit the training data.

* Minimum Node Records. This is the minimum number of training patterns that a
given node is responsible for. Once at or below this minimum, we must stop splitting and
adding new nodes. Nodes that account for too few training patterns are expected to be
too specific and are likely to overfit the training data.


These two approaches will be user-specified arguments to our tree building procedure. There
is one more condition; it is possible to choose a split in which all rows belong to one group. In
this case, we will be unable to continue splitting and adding child nodes as we will have no
records to split on one side or another.

In [134]:
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return


    # if len(set([row[-1] for row in left]))==1:
    #     print('LEFT PURE at depth {0}'.format(depth))
        
    # if len(set([row[-1] for row in right]))==1:
    #     print('RIGHT PURE at depth {0}'.format(depth))   

    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_size or len(set([row[-1] for row in left]))==1:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)       
        split(node['left'], max_depth, min_size, depth+1)
        
    # process right child
    if len(right) <= min_size or len(set([row[-1] for row in right]))==1:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)


# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root   

# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('depth:%s %s[X%d < %.3f]' % ((depth,depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('depth:%s %s[%s]' % ((depth,depth*' ',str(node)+'_Leaf')))    

In [89]:
dataset = [[2.771244718,1.784783929,0],
[1.728571309,1.169761413,0],
[3.678319846,2.81281357,0],
[3.961043357,2.61995032,0],
[3.961043327,2.61995032,0],
[3.961043357,2.61995032,0],
[2.999208922,2.209014212,0],
[7.497545867,3.162953546,1],
[9.00220326,3.339047188,1],
[7.444542326,0.476683375,1],
[10.12493903,3.234550982,1],
[6.642287351,3.319983761,1]]
tree = build_tree(dataset, 3, 1)
print_tree(tree)

left : 1
right : 1
depth:0 [X1 < 6.642]
depth:1  [0_Leaf]
depth:1  [1_Leaf]


In [90]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [92]:
# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

In [137]:
# Test CART on Bank Note dataset
seed(1)
# load and prepare data
filepath = '/Users/maheshwars/Desktop/venv/data'
filename = filepath +'/banknotes_data.csv'
dataset = load_csv(filename)
print('Loaded data file {0} with {1} rows and {2} columns'.format(filename, len(dataset),
                                                                  len(dataset[0])))

# convert string attributes to integers
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
    
# evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Loaded data file /Users/maheshwars/Desktop/venv/data/banknotes_data.csv with 1370 rows and 5 columns
Scores: [96.35036496350365, 95.25547445255475, 96.71532846715328, 97.8102189781022, 97.44525547445255]
Mean Accuracy: 96.715%


# __Naive Bayes Classifier__

In [1]:
# Example of summarizing a dataset
from math import sqrt

# Calculate the mean of a list of numbers
def mean(numbers):
    return sum(numbers)/float(len(numbers))
    
# Calculate the standard deviation of a list of numbers
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers)-1)
    return sqrt(variance)

# Calculate the mean, stdev and count for each column in a dataset
def summarize_dataset(dataset):
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    del(summaries[-1])
    return summaries

# Test summarizing a dataset
dataset = [[3.393533211,2.331273381,0],
[3.110073483,1.781539638,0],
[1.343808831,3.368360954,0],
[3.582294042,4.67917911,0],
[2.280362439,2.866990263,0],
[7.423436942,4.696522875,1],
[5.745051997,3.533989803,1],
[9.172168622,2.511101045,1],
[7.792783481,3.424088941,1],
[7.939820817,0.791637231,1]]


summary = summarize_dataset(dataset)
print(summary)

[(5.1783333865, 2.7665845055177263, 10), (2.9984683241, 1.218556343617447, 10)]


In [2]:
for column in zip(*dataset): #dataset creates a list of all rows , and zip groupbys the rows by index, similar to what transpose does to a table
    print(column)

(3.393533211, 3.110073483, 1.343808831, 3.582294042, 2.280362439, 7.423436942, 5.745051997, 9.172168622, 7.792783481, 7.939820817)
(2.331273381, 1.781539638, 3.368360954, 4.67917911, 2.866990263, 4.696522875, 3.533989803, 2.511101045, 3.424088941, 0.791637231)
(0, 0, 0, 0, 0, 1, 1, 1, 1, 1)


In [3]:
# Split the dataset by class values, returns a dictionary
def separate_by_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if (class_value not in separated):
            separated[class_value] = list()
        separated[class_value].append(vector)
    return separated

def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    separated_summaries = dict()
    for class_value, rows in separated.items():
        separated_summaries[class_value] = summarize_dataset(rows)
    return separated_summaries

## Gaussian Probability density function

Calculating the probability or likelihood of observing a given real-value like X1 is diﬃcult. One
way we can do this is to assume that X1 values are drawn from a distribution, such as a bell
curve or Gaussian distribution.
A Gaussian distribution can be summarized using only two numbers: the mean and the
standard deviation.

In [4]:
# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

### Class Probabilities

Now it is time to use the statistics calculated from our training data to calculate probabilities
for new data. Probabilities are calculated separately for each class. This means that we first
calculate the probability that a new piece of data belongs to the first class, then calculate
probabilities that it belongs to the second class, and so on for all the classes. The probability
that a piece of data belongs to a class is calculated as follows:

P (class|data) = P (X|class) ×P (class) 


You may note that this is diﬀerent from the Bayes Theorem described above. The division
have been removed to simplify the calculation. This means that the result is no longer strictly a
probability of the data belonging to a class. The value is still maximized, meaning that the
calculation for the class that results in the largest value is taken as the prediction. This is a
common implementation simplification as we are often more interested in the class prediction
rather than the probability.

P (class = 0|X1, X2) = P (X1|class = 0) ×P (X2|class = 0) ×P (class = 0) 

In [5]:
from math import sqrt
from math import pi
from math import exp

def calculate_class_probabilities_ms(summaries,data):
    p_class = []
    for d_class,d_stats in summaries.items():
        prob = d_stats[0][2]/sum([stat[0][2] for key,stat in summaries.items()]) #inefficient
        for i in range(len(data)-1):
            prob *= calculate_probability(data[i],d_stats[i][0],d_stats[i][1])
        p_class.append(prob)
    return p_class    

# Calculate the probabilities of predicting each class for a given row
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    probabilities = dict()
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities
    
# Predict the class for a given row
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

# Naive Bayes Algorithm
def naive_bayes(train, test):
    summarize = summarize_by_class(train)
    predictions = list()
    for row in test:
        output = predict(summarize, row)
        predictions.append(output)
    return(predictions)

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores    

# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
        
# Test calculating class probabilities
dataset = [[3.393533211,2.331273381,0],
[3.110073483,1.781539638,0],
[1.343808831,3.368360954,0],
[3.582294042,4.67917911,0],
[2.280362439,2.866990263,0],
[7.423436942,4.696522875,1],
[5.745051997,3.533989803,1],
[9.172168622,2.511101045,1],
[7.792783481,3.424088941,1],
[7.939820817,0.791637231,1]]

summaries = summarize_by_class(dataset)
probabilities = calculate_class_probabilities(summaries, dataset[0])
print(probabilities)


{0: 0.05032427673372076, 1: 0.00011557718379945765}


In [11]:

seed(1)
# load and prepare data
filepath = '/Users/maheshwars/Desktop/venv/data/iris'
filename = filepath +'/iris_data.csv'
dataset = load_csv(filename)
print('Loaded data file {0} with {1} rows and {2} columns'.format(filename, len(dataset),
                                                                  len(dataset[0])))

for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
    
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

# evaluate algorithm
n_folds = 5
scores = evaluate_algorithm(dataset, naive_bayes, n_folds)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Loaded data file /Users/maheshwars/Desktop/venv/data/iris/iris_data.csv with 150 rows and 5 columns
Scores: [93.33333333333333, 96.66666666666667, 100.0, 93.33333333333333, 93.33333333333333]
Mean Accuracy: 95.333%


# __K_Nearest Neighbours__

The k-Nearest Neighbors algorithm or KNN for short is a very simple technique. The entire
training dataset is stored. When a prediction is required, the k-most similar records to a
new record from the training dataset are then located. From these neighbors, a summarized
prediction is made. Similarity between records can be measured many diﬀerent ways. A problem
or data-specific method can be used. Generally, with tabular data, a good starting point is the
Euclidean distance.
Once the neighbors are discovered, the summary prediction can be made by returning the
most common outcome or taking the average. As such, KNN can be used for classification
or regression problems. There is no model to speak of other than holding the entire training
dataset. Because no work is done until a prediction is required, KNN is often referred to as a
lazy learning method.

In [101]:
# k-nearest neighbors on the Abalone Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
    
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


    
# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)
    
# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors


### Abalone Case Study as Classification

In [ ]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
    
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted) #accuracy for classification
        scores.append(accuracy)
    return scores

# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

# kNN Algorithm
def k_nearest_neighbors(train, test, num_neighbors):
    predictions = list()
    for row in test:
        output = predict_classification(train, row, num_neighbors)
        predictions.append(output)
    return (predictions)    

In [16]:
# Test the kNN on the Abalone dataset

seed(1)
# load and prepare data
filepath = '/Users/maheshwars/Desktop/venv/data/abalone'
filename = filepath +'/abalone_data.csv'
dataset = load_csv(filename)
print('Loaded data file {0} with {1} rows and {2} columns'.format(filename, len(dataset),
                                                                  len(dataset[0])))

for i in range(1, len(dataset[0])):
    str_column_to_float(dataset, i)

# convert first column to integers
str_column_to_int(dataset, 0)


# evaluate algorithm
n_folds = 5
num_neighbors = 5
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Loaded data file /Users/maheshwars/Desktop/venv/data/abalone/abalone_data.csv with 4177 rows and 9 columns
Scores: [24.790419161676645, 21.79640718562874, 23.592814371257482, 21.676646706586826, 23.353293413173652]
Mean Accuracy: 23.042%


### Abalone Case Study as Regression

In [23]:
# Calculate root mean squared error
def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error ** 2)
    mean_error = sum_error / float(len(actual))
    return sqrt(mean_error)
    
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        rmse = rmse_metric(actual, predicted) # rmse for regression
        scores.append(rmse)
    return scores


# Make a prediction with neighbors
def predict_regression(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = sum(output_values) / float(len(output_values))
    return prediction
    
# kNN Algorithm
def k_nearest_neighbors(train, test, num_neighbors):
    predictions = list()
    for row in test:
        output = predict_regression(train, row, num_neighbors)
        predictions.append(output)
    return(predictions)
    

In [24]:
# Test the kNN on the Abalone dataset

seed(1)
# load and prepare data
filepath = '/Users/maheshwars/Desktop/venv/data/abalone'
filename = filepath +'/abalone_data.csv'
dataset = load_csv(filename)
print('Loaded data file {0} with {1} rows and {2} columns'.format(filename, len(dataset),
                                                                  len(dataset[0])))

for i in range(1, len(dataset[0])):
    str_column_to_float(dataset, i)

# convert first column to integers
str_column_to_int(dataset, 0)


# evaluate algorithm
n_folds = 5
num_neighbors = 5
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean RMSE: %.3f' % (sum(scores)/float(len(scores))))

Loaded data file /Users/maheshwars/Desktop/venv/data/abalone/abalone_data.csv with 4177 rows and 9 columns
Scores: [2.170383116929243, 2.2087035241256405, 2.2321118594939215, 2.4013070293283603, 2.2274928845898017]
Mean RMSE: 2.248


# __Learning Vector Quantization__

A limitation of k-Nearest Neighbors is that you must keep a large database of training examples
in order to make predictions. The Learning Vector Quantization algorithm addresses this by
learning a much smaller subset of patterns that best represent the training data.

The Learning Vector Quantization (LVQ) algorithm is a lot like k-Nearest Neighbors. __Predictions
are made by finding the best match among a library of patterns__. The diﬀerence is that the library
of patterns is learned from training data, rather than using the training patterns themselves.

The library of patterns is called __codebook vectors__ and each pattern is called a codebook.
The codebook vectors are initialized to randomly selected values from the training dataset.
Then, over a number of epochs, __they are adapted to best summarize the training data using a
learning algorithm. The learning algorithm shows one training record at a time, finds the best
matching unit among the codebook vectors and moves it closer to the training record if they
have the same class, or further away if they have diﬀerent classes.__

Once prepared, the codebook vectors are used to make predictions using the k-Nearest
Neighbors algorithm where **k=1**. The algorithm was developed for classification predictive
modeling problems, but can be adapted for use with regression problems.

### Best Matching Unit
The Best Matching Unit or BMU is the codebook vector that is most similar to a new piece of
data. To locate the BMU for a new piece of data within a dataset we must first calculate the
distance between each codebook to the new piece of data. We can do this using our distance
function above. Once distances are calculated, we must sort all of the codebooks by their
distance to the new data. We can then return the first or most similar codebook vector.

We can do this by keeping track of the distance for each record in the dataset as a tuple,
sort the list of tuples by the distance (in descending order) and then retrieve the BMU. Below
is a function named get best matching unit() that implements this.

In [28]:
# Example of getting the BMU
from math import sqrt

# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

# Locate the best matching unit
def get_best_matching_unit(codebooks, test_row):
    distances = list()
    for codebook in codebooks:
        dist = euclidean_distance(codebook, test_row)
        distances.append((codebook, dist))
    distances.sort(key=lambda tup: tup[1])
    return distances[0][0]

# Test best matching unit function
dataset = [[2.7810836,2.550537003,0],
[1.465489372,2.362125076,0],
[3.396561688,4.400293529,0],
[1.38807019,1.850220317,0],
[3.06407232,3.005305973,0],
[7.627531214,2.759262235,1],
[5.332441248,2.088626775,1],
[6.922596716,1.77106367,1],
[8.675418651,-0.242068655,1],
[7.673756466,3.508563011,1]]
test_row = dataset[0]
bmu = get_best_matching_unit(dataset, test_row)
print(bmu)   

[2.7810836, 2.550537003, 0]


### Training Codebook Vectors
The first step in training a set of codebook vectors is to initialize the set. We can initialize it with
patterns constructed from random features in the training dataset. Below is a function named
random codebook() that implements this. Random input and output features are selected from
the training data.

In [33]:
# Create a random codebook vector
def random_codebook(train):
    n_records = len(train)
    n_features = len(train[0])
    codebook = [train[randrange(n_records)][i] for i in range(n_features)]
    return codebook

The best matching unit is found for each training pattern and only this best matching unit
is updated. The diﬀerence between the training pattern and the BMU is calculated as the error.
The class values (assumed to be the last value in the list) are compared. If they match, the
error is added to the BMU to bring it closer to the training pattern, otherwise it is subtracted
to push it further away.

The amount that the BMU is adjusted is controlled by a __learning rate__. This is a weighting
on the amount of change made to all BMUs. For example, a learning rate of 0.3 means that
BMUs are only moved by 30% of the error or diﬀerence between training patterns and BMUs.
Further, the learning rate is adjusted so that it has maximum eﬀect in the first epoch and
less eﬀect as training continues until it has a minimal eﬀect in the final epoch. This is called a
**linear decay learning rate schedule** and can also be used in artificial neural networks. We can
summarize this decay in learning rate by epoch number as follows:

**rate = learning rate × (1.0 − (epoch / total_epochs))**

In [36]:
# Train a set of codebook vectors
def train_codebooks(train, n_codebooks, lrate, epochs):
    codebooks = [random_codebook(train) for i in range(n_codebooks)]
    for epoch in range(epochs):
        rate = lrate * (1.0-(epoch/float(epochs)))
        sum_error = 0.0
        for row in train:
            bmu = get_best_matching_unit(codebooks, row)
            for i in range(len(row)-1):
                error = row[i] - bmu[i]
                sum_error += error**2
                if bmu[-1] == row[-1]:
                    bmu[i] += rate * error
                else:
                    bmu[i] -= rate * error
        #print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, rate, sum_error))
    return codebooks

In [38]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

### Training and prediction on ionosphere dataset

In [60]:
# Make a prediction with codebook vectors
def predict(codebooks, test_row):
    bmu = get_best_matching_unit(codebooks, test_row)
    return bmu[-1]

# LVQ Algorithm
def learning_vector_quantization(train, test, n_codebooks, lrate, epochs):
    codebooks = train_codebooks(train, n_codebooks, lrate, epochs)
    predictions = list()
    for row in test:
        output = predict(codebooks, row)
        predictions.append(output)
    return(predictions)
    
# Test LVQ on Ionosphere dataset
seed(1)

# load and prepare data
filepath = '/Users/maheshwars/Desktop/venv/data/ionosphere'
filename = filepath +'/ionosphere_data.csv'
dataset = load_csv(filename)
print('Loaded data file {0} with {1} rows and {2} columns'.format(filename, len(dataset),
                                                                  len(dataset[0])))

for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)

# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

# evaluate algorithm
# for i in range(1,15,2):
n_folds = 5
learn_rate = 0.3
n_epochs = 50
n_codebooks = 20
scores = evaluate_algorithm(dataset, learning_vector_quantization, n_folds, n_codebooks,
learn_rate, n_epochs)
print('Lrate :{1}, learning rate Scores: {0}'.format(scores,learn_rate))
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))    

Loaded data file /Users/maheshwars/Desktop/venv/data/ionosphere/ionosphere_data.csv with 351 rows and 35 columns
Lrate :0.3, learning rate Scores: [88.57142857142857, 90.0, 88.57142857142857, 88.57142857142857, 80.0]
Mean Accuracy: 87.143%


# __Backpropagation__

The principle of the backpropagation approach is to model a given function by modifying
internal weightings of input signals to produce an expected output signal. The system is trained
using a supervised learning method where the error between the system’s output and a known
expected output is presented to the system and used to modify its internal state.

Technically, the backpropagation algorithm is a method for training the weights in a multilayer
feedforward neural network. As such, it requires a network structure to be defined of one or
more layers where one layer is fully connected to the next layer.

## Initialize Network
Let’s start with something easy: the creation of a new network ready for training. Each neuron
has a set of weights that need to be maintained. One weight for each input connection and an
additional weight for the bias. We will need to store additional properties for a neuron during
training, therefore **we will use a dictionary to represent each neuron and store properties by
names such as weights for the weights**.
A network is organized into layers. The input layer is really just a row from our training
dataset. The first real layer is the hidden layer. This is followed by the output layer that has
one neuron for each class value. **We will organize layers as arrays of dictionaries and treat the
whole network as an array of layers**. It is good practice to initialize the network weights to
small random numbers. In this case, will we use random numbers in the range of 0 to 1.

Below is a function named initialize network() that creates a new neural network ready
for training. It accepts three parameters: the number of inputs, the number of neurons to have
in the hidden layer and the number of outputs. You can see that for the hidden layer we create
n hidden neurons and **each neuron** in the hidden layer has **n inputs + 1 weights**, one for each
input column in a dataset and an additional one for the bias.

You can also see that the output layer that connects to the hidden layer has n outputs
neurons, **each neuron with n hidden + 1 weights**. This means that each neuron in the output layer
connects to (has a weight for) each neuron in the hidden layer.

In [75]:

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in
    range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in
    range(n_outputs)]
    network.append(output_layer)
    return network

## Forward-Propagate

We can break forward-propagation down into three parts:

1. __Neuron Activation:__

   
    Neuron activation is calculated as the weighted sum of the inputs. Much like linear regression.


         activation = bias + summation{ weights[i] * inputs[i] }

      
3. __Neuron Transfer:__

   
    Once a neuron is activated, we need to transfer the activation to see what the neuron output
    actually is. Diﬀerent transfer functions can be used. It is traditional to use the sigmoid activation
    function, but you can also use the tanh (hyperbolic tangent) function to transfer outputs. More
    recently, the rectifier transfer function has been popular with large deep learning networks.
   
5. __Forward-Propagation:__
    
    We work through each layer of our network
    calculating the outputs for each neuron. All of the outputs from one layer become inputs to the
    neurons on the next layer. 

In [77]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1] #bias column
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))    

# Forward-propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs    


## Backpropagate Error
The backpropagation algorithm is named for the way in which weights are trained. Error is
calculated between the expected outputs and the outputs forward-propagated from the network.
These **errors are then propagated backward** through the network from the output layer to the
hidden layer, **assigning blame for the error** and updating weights as they go. 

### Transfer Derivative
Given an output value from a neuron, we need to calculate it’s slope. We are using the sigmoid
transfer function, the derivative of which can be calculated as follows:

    derivative= output ×(1.0−output)

In [79]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

### Error Backpropagation

The first step is to calculate the __error for each output neuron__; this will give us our __error signal
(input) to propagate backwards through the network__. The error for a given neuron can be
calculated as follows:

    error = (expected−output) ×transfer derivative(output)


Where expected is the expected output value for the neuron, output is the output value
for the neuron and transfer_derivative() calculates the slope of the neuron’s output value,
as shown above. This error calculation is used for neurons in the output layer. The expected
value is the class value itself. In the hidden layer, things are a little more complicated.


The error signal for a neuron in the hidden layer is calculated as the weighted error of each
neuron in the output layer. Think of the error traveling back along the weights of the output
layer to the neurons in the hidden layer. The backpropagated error signal is accumulated and
then used to determine the error for the neuron in the hidden layer, as follows:

    error = (weightk ×errorj ) ×transfer derivative(output) 


Where error j is the error signal from the jth neuron in the output layer, weight k is the
weight that connects the kth neuron to the current neuron(neuron of hidden layer) and output is the output for the
current neuron. Below is a function named backward propagate error() that implements this
procedure.

You can see that the error signal calculated for each neuron is stored with the name delta.
You can see that the layers of the network are iterated in reverse order, starting at the output
and working backwards. This ensures that the neurons in the output layer have delta values
calculated first that neurons in the hidden layer can use in the subsequent iteration.
You can see that **the error signal for neurons in the hidden layer is accumulated from neurons
in the output layer where the hidden neuron number j is also the index of the neuron’s weight
in the output layer neuron[’weights’][j].**

In [80]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [86]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

### Train Network
As mentioned, the network is updated using stochastic gradient descent. This involves first
looping for a fixed number of epochs and within each epoch updating the network for each row
in the training dataset.

Because updates are made for each training pattern, this type of learning is called online
learning. If errors were accumulated across an epoch before updating the weights, this is called
batch learning or batch gradient descent. Below is a function that implements the training of
an already initialized neural network with a given training dataset, learning rate, fixed number
of epochs and an expected number of output values.

The expected number of output values is used to transform class values in the training data
into a one hot encoding. That is a binary vector with one column for each class value to match
the output of the network. This is required to calculate the error for the output layer. You can
also see that the sum squared error between the expected output and the network output is
accumulated each epoch and printed. This is helpful to create a trace of how much the network
is learning and improving each epoch.

In [104]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        #print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [85]:
# Example of training a network by backpropagation
from math import exp
from random import seed
from random import random

# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
[1.465489372,2.362125076,0],
[3.396561688,4.400293529,0],
[1.38807019,1.850220317,0],
[3.06407232,3.005305973,0],
[7.627531214,2.759262235,1],
[5.332441248,2.088626775,1],
[6.922596716,1.77106367,1],
[8.675418651,-0.242068655,1],
[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
    print(layer)

>epoch=0, lrate=0.500, error=6.371
>epoch=1, lrate=0.500, error=5.584
>epoch=2, lrate=0.500, error=5.331
>epoch=3, lrate=0.500, error=5.314
>epoch=4, lrate=0.500, error=5.330
>epoch=5, lrate=0.500, error=5.341
>epoch=6, lrate=0.500, error=5.346
>epoch=7, lrate=0.500, error=5.347
>epoch=8, lrate=0.500, error=5.347
>epoch=9, lrate=0.500, error=5.346
>epoch=10, lrate=0.500, error=5.345
>epoch=11, lrate=0.500, error=5.343
>epoch=12, lrate=0.500, error=5.342
>epoch=13, lrate=0.500, error=5.340
>epoch=14, lrate=0.500, error=5.339
>epoch=15, lrate=0.500, error=5.337
>epoch=16, lrate=0.500, error=5.335
>epoch=17, lrate=0.500, error=5.334
>epoch=18, lrate=0.500, error=5.332
>epoch=19, lrate=0.500, error=5.330
[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.99157530623528, 'delta': -0.0005341455083516953}, {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381], 'output': 0.9844051047537846, 'delta': 0.0011954440011118744}]
[{'weights': [0

In [105]:
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return(predictions)

In [116]:
# Test Backprop on Seeds dataset
seed(1)

def load_txt(filename):
    dataset = []
    with open(filename, "r") as file:
        for line in file:
            # Split by spaces and convert to floats (or int, if applicable)
            values = list(line.strip().split())
            dataset.append(values)
    return dataset

# load and prepare data
filepath = '/Users/maheshwars/Desktop/venv/data/'
filename = filepath +'/seeds_data.txt'
dataset = load_txt(filename)
print('Loaded data file {0} with {1} rows and {2} columns'.format(filename, len(dataset),
                                                                  len(dataset[0])))

print(dataset[0])
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)


# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)


print(dataset[0])

Loaded data file /Users/maheshwars/Desktop/venv/data//seeds_data.txt with 210 rows and 8 columns
['15.26', '14.84', '0.871', '5.763', '3.312', '2.221', '5.22', '1']
[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, 2]
[0.4409820585457979, 0.5020661157024793, 0.570780399274047, 0.48648648648648646, 0.48610121168923714, 0.18930164220052273, 0.3451501723289019, 2]


In [119]:

# evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 700
n_hidden = 8

scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [92.85714285714286, 95.23809523809523, 95.23809523809523, 95.23809523809523, 92.85714285714286]
Mean Accuracy: 94.286%
